In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
import os
os.chdir('/content/drive/MyDrive/AI_Advanced_CK/NLP/3_Paragraph_classification')

# Import Toolkit

In [41]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.utils import to_categorical
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

# Dataset

In [42]:
# # Đường dẫn đến file txt
# file_txt_path = "./news_categories/news_categories.txt"
# with open(file_txt_path, 'r', encoding='utf-8') as file:
#     content = file.read()

# print(content[0:10000])

# lines = content.strip().split('\n')
# labels = []
# sentences = []
# for line in lines:
#     label, sentence = line.split(' ', 1)
#     labels.append(label)
#     sentences.append(sentence)

In [43]:
# print("Labels:", labels[:5])
# print("Sentences:", sentences[:5])

In [44]:
import pandas as pd

# Đường dẫn tới file CSV (thay đổi theo đường dẫn thực tế của bạn)
train_csv_path = './train.csv'
test_csv_path = './test.csv'

# Đọc dữ liệu từ file CSV
train_data = pd.read_csv(train_csv_path)
test_data = pd.read_csv(test_csv_path)

# Khởi tạo các danh sách để lưu dữ liệu
train_sentences = []
test_sentences = []
train_labels = []
test_labels = []

# Lưu dữ liệu từ train.csv vào các danh sách
for index, row in train_data.iterrows():
    train_sentences.append(row['Title'] + " " + row['Description'])
    train_labels.append(row['Class Index'])

# Lưu dữ liệu từ test.csv vào các danh sách
for index, row in test_data.iterrows():
    test_sentences.append(row['Title'] + " " + row['Description'])
    test_labels.append(row['Class Index'])

# Hiển thị kết quả để kiểm tra
print(f'Train sentences: {train_sentences[:3]}')  # Hiển thị 3 câu đầu tiên để kiểm tra
print(f'Train labels: {train_labels[:3]}')        # Hiển thị 3 nhãn đầu tiên để kiểm tra
print(f'Test sentences: {test_sentences[:3]}')    # Hiển thị 3 câu đầu tiên để kiểm tra
print(f'Test labels: {test_labels[:3]}')          # Hiển thị 3 nhãn đầu tiên để kiểm tra

Train sentences: ["Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.', "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums."]
Train labels: [3, 3, 3]
Test sentences: ["Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.", 'The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com) SPACE.com - TORONTO, Canada

In [45]:
print(len(train_sentences))
print(len(train_labels))

120000
120000


In [46]:
# # Thêm phần tử mới vào train_sentences và train_labels
# # s = 'Nguyen Van Manh'
# s = 'Nguyễn Văn Mạnh'
# train_sentences.append(s)
# l = 1
# train_labels = np.append(train_labels, l)

# print(train_sentences[-1])
# print(train_labels[-1])

In [47]:
print(len(train_sentences))
print(len(train_labels))
print(train_sentences[0])
print(train_labels[0])
print(train_labels) # label chỉ có 2 nhãn : 0 và 1

120000
120000
Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
3
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4

In [48]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

## Parameters

In [49]:
vocab_size = 10000
embedding = 64
max_length = 256
embedding_dim = 128  # Kích thước vector nhúng
units = 128

## Tokenizer

In [50]:
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# # Thử nghiệm với các giá trị khác nhau của vocab_size
# vocab_sizes = [5000, 10000, 20000]

# for vocab_size in vocab_sizes:
#     tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
#     tokenizer.fit_on_texts(train_sentences)

#     # Chuyển đổi các câu thành các chuỗi số
#     train_sequences = tokenizer.texts_to_sequences(train_sentences)

#     # Đệm các chuỗi để có độ dài bằng nhau
#     padded_sequences = pad_sequences(train_sequences, maxlen=100, padding='post')

#     print(f"Vocabulary Size: {vocab_size}")
#     print(f"Example of padded sequences: {padded_sequences[:2]}")
#     print(f"Number of unique tokens: {len(tokenizer.word_index)}")
#     print("-" * 50)

In [51]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer (num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts (train_sentences + test_sentences + ['Nguyễn Văn Mạnh'])

In [52]:
train_sentences_seq = tokenizer.texts_to_sequences(train_sentences)
padded_train_sentences = pad_sequences (train_sentences_seq, maxlen=max_length, truncating='post', padding='post')

In [53]:
word_index = tokenizer.word_index
len(tokenizer.word_index)

71999

In [54]:
# print(word_index['romance'])
# print(word_index['nguyen'])
# print(word_index['van'])
# # đầu vào là 'Nguyễn Văn Mạnh'
# print(word_index['nguyễn'])
# print(word_index['văn'])
# print(word_index['mạnh']) # sẽ chuyển thành in thường hết

In [55]:
# # Từ index lấy ra word
# reverse_word_index = {index: word for word, index in word_index.items()}
# print(reverse_word_index[900])
# print(reverse_word_index[28201])
# print(reverse_word_index[1090])
# print(reverse_word_index[65341])
# print(reverse_word_index[65342])
# print(reverse_word_index[65343])

In [56]:
test_sentences_seq = tokenizer.texts_to_sequences (test_sentences)
padded_test_sentences = pad_sequences(test_sentences_seq, maxlen=max_length, truncating='post', padding='post')

In [57]:
print(len(test_sentences[0]))
print(test_sentences[0])

161
Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.


In [58]:
padded_test_sentences

array([[ 912,    9,  138, ...,    0,    0,    0],
       [   2,  475,   17, ...,    0,    0,    0],
       [   1,   58,  425, ...,    0,    0,    0],
       ...,
       [2204, 1805, 3053, ...,    0,    0,    0],
       [ 147,    5, 2672, ...,    0,    0,    0],
       [1551,  561,   65, ...,    0,    0,    0]], dtype=int32)

In [59]:
padded_test_sentences.shape

(7600, 256)

In [60]:
print(len(padded_test_sentences[0]))

256


## Label Encoder

In [61]:
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

In [62]:
num_classes = len(np.unique(train_labels_encoded))
num_classes

4

In [63]:
# Chuyển nhãn sang dạng one-hot encoding
train_labels_one_hot = to_categorical(train_labels_encoded, num_classes=num_classes)
test_labels_one_hot = to_categorical(test_labels_encoded, num_classes=num_classes)

In [64]:
len(label_encoder.classes_)

4

# Model

## Model1

In [65]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Bidirectional, Dense, Flatten
model1 = Sequential()
model1.add(Embedding(vocab_size, embedding_dim, input_length = max_length)) # embedding ở đây chính là output_dim
model1.add(Flatten())
# model.add(LSTM(embedding)) # embedding node ẩn ???
model1.add(Dense(128, activation='relu'))
model1.add(Dense(num_classes, activation='softmax'))

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 256, 128)          1280000   
                                                                 
 flatten_1 (Flatten)         (None, 32768)             0         
                                                                 
 dense_8 (Dense)             (None, 128)               4194432   
                                                                 
 dense_9 (Dense)             (None, 4)                 516       
                                                                 
Total params: 5474948 (20.89 MB)
Trainable params: 5474948 (20.89 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## RNN

In [66]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding

rnn_model  = Sequential()
rnn_model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
rnn_model.add(SimpleRNN(units))
rnn_model.add(Dense(128, activation='relu'))
rnn_model.add(Dense(num_classes, activation='softmax'))

rnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
rnn_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 256, 128)          1280000   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dense_10 (Dense)            (None, 128)               16512     
                                                                 
 dense_11 (Dense)            (None, 4)                 516       
                                                                 
Total params: 1329924 (5.07 MB)
Trainable params: 1329924 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## LSTM

In [67]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

lstm_model = Sequential()
lstm_model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
lstm_model.add(LSTM(units))
lstm_model.add(Dense(128, activation='relu'))
lstm_model.add(Dense(num_classes, activation='softmax'))

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lstm_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 256, 128)          1280000   
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584    
                                                                 
 dense_12 (Dense)            (None, 128)               16512     
                                                                 
 dense_13 (Dense)            (None, 4)                 516       
                                                                 
Total params: 1428612 (5.45 MB)
Trainable params: 1428612 (5.45 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [68]:
# # Callbacks
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
# checkpoint = ModelCheckpoint('./Phuc_Save_Model/LSTM_model.h5', monitor='val_loss', verbose=1, save_best_only=True)

# # Fit the model
# H = lstm_model.fit(padded_train_sentences, train_labels, epochs=10, batch_size=128,
#                             validation_data=(padded_test_sentences, test_labels),
#                             callbacks=[early_stopping, checkpoint])

# # Accessing training history
# print("Training History:", H.history)

In [69]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense, Dropout

# model = Sequential()
# model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
# model.add(LSTM(64, return_sequences=True, activation='relu'))
# model.add(LSTM(128, return_sequences=True, activation='relu'))
# model.add(LSTM(64, return_sequences=False, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

# optimizer = Adam(learning_rate=0.001)
# model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])

# model.summary()

In [70]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense, Dropout

# model = Sequential()
# model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(units=128))
# model.add(Dropout(0.2))
# model.add(Dense(10, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# model.summary()

In [71]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, LSTM, Dense

# model = Sequential()
# model.add(Embedding(vocab_size, embedding, input_length=max_length))
# model.add(LSTM(64))  # LSTM layer with 64 units
# model.add(Dense(10, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# model.summary()

## BiLSTM

In [72]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense

# Create BiLSTM model
bilstm_model = Sequential()
bilstm_model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
bilstm_model.add(Bidirectional(LSTM(256)))

bilstm_model.add(Dense(128, activation='relu'))
bilstm_model.add(Dense(num_classes, activation='softmax'))

bilstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
bilstm_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 256, 128)          1280000   
                                                                 
 bidirectional_1 (Bidirecti  (None, 512)               788480    
 onal)                                                           
                                                                 
 dense_14 (Dense)            (None, 128)               65664     
                                                                 
 dense_15 (Dense)            (None, 4)                 516       
                                                                 
Total params: 2134660 (8.14 MB)
Trainable params: 2134660 (8.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Training

In [77]:
# model = model1
# model = rnn_model
model = lstm_model
# model = bilstm_model

In [78]:
print(padded_train_sentences.shape)
print(padded_test_sentences.shape)
print(train_labels_one_hot.shape)
print(test_labels_one_hot.shape)

(120000, 256)
(7600, 256)
(120000, 4)
(7600, 4)


In [ ]:
# Callbacks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
checkpoint = ModelCheckpoint('./Manh_Save/LSTM_model.h5', monitor='val_loss', verbose=1, save_best_only=True)

# Fit the model
H = model.fit(padded_train_sentences, train_labels_one_hot, epochs=10, batch_size=128,
                            validation_data=(padded_test_sentences, test_labels_one_hot),
                            callbacks=[early_stopping, checkpoint])

# Accessing training history
print("Training History:", H.history)

Epoch 1/10
938/938 [==============================] - ETA: 0s - loss: 1.3865 - accuracy: 0.2501
Epoch 1: val_loss improved from inf to 1.38653, saving model to ./Manh_Save/LSTM_model.h5
938/938 [==============================] - 1000s 1s/step - loss: 1.3865 - accuracy: 0.2501 - val_loss: 1.3865 - val_accuracy: 0.2500
Epoch 2/10


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


938/938 [==============================] - ETA: 0s - loss: 1.3864 - accuracy: 0.2494
Epoch 2: val_loss improved from 1.38653 to 1.38630, saving model to ./Manh_Save/LSTM_model.h5
938/938 [==============================] - 959s 1s/step - loss: 1.3864 - accuracy: 0.2494 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 3/10
938/938 [==============================] - ETA: 0s - loss: 1.3863 - accuracy: 0.2490
Epoch 3: val_loss did not improve from 1.38630
938/938 [==============================] - 923s 984ms/step - loss: 1.3863 - accuracy: 0.2490 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 4/10
757/938 [=======================>......] - ETA: 2:51 - loss: 1.3863 - accuracy: 0.2476

In [ ]:
train_acc = H.history['accuracy']
train_loss = H.history['loss']

val_acc = H.history['val_accuracy']
val_loss = H.history['val_loss']

index_loss = np.argmin(val_loss)
index_acc = np.argmax(val_acc)

val_lowest = val_loss[index_loss]
val_highest = val_acc[index_acc]

Epochs = [i+1 for i in range(len(train_acc))]

loss_label = f'Best Epoch = {str(index_loss + 1)}'
acc_label = f'Best Epoch = {str(index_acc + 1)}'

plt.figure(figsize= (20,8))
plt.style.use('fivethirtyeight')

plt.subplot(1,2,1)
plt.plot(Epochs , train_loss , 'r', label = 'Training Loss')
plt.plot(Epochs , val_loss , 'g' , label = 'Validation Loss')
plt.scatter(index_loss +1 , val_lowest , s = 150 , c = 'blue' , label = loss_label)
plt.title('Training vs Validation (loss)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(Epochs , train_acc , 'r', label= 'Training Accuracy')
plt.plot(Epochs , val_acc , 'g' , label = 'Validation Accuracy')
plt.scatter(index_acc + 1 , val_highest , s= 150 , c = 'blue' , label= acc_label)
plt.title('Training vs Validation (Accuracy)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout
plt.show()

# Test

In [ ]:
test_sen = ['Nguyễn Văn Mạnh']
test_sen = ['hà_nội t t chốt danh_sách dự afc cup thể_thao việt_nam zing vn tiền_vệ có lối chơi sáng_tạo bậc nhất của hà_nội t t benicio được hlv']
test_seq = tokenizer.texts_to_sequences (test_sen)
padded_test_seq=pad_sequences(test_seq, maxlen = max_length, truncating='post', padding='post')
# padded_test_seq

print(padded_test_seq.shape)

import numpy as np
# Dự đoán nhãn cho tập kiểm tra
predicted_prob = model.predict(padded_test_seq)
# Chuyển đổi nhãn dự đoán từ one-hot encoding về dạng số
predicted_labels_encoded = np.argmax(predicted_prob, axis=1)
# Chuyển đổi nhãn từ dạng số về dạng chuỗi
predicted_labels = label_encoder.inverse_transform(predicted_labels_encoded)
# In ra một số nhãn dự đoán
print("Predicted labels (first 10):", predicted_labels[:10])

## WEIGHT

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
weights.shape # (10000, 64) thì 64 chính là embeding

In [ ]:
# Tìm trọng số của một từ cụ thể trong từ điển
position = tokenizer.texts_to_sequences (['Mạnh'])
weights[position]

## Embedings Từ

In [ ]:
word = 'nguyễn'

modelx = Sequential()
modelx.add(Embedding(vocab_size, embedding_dim, input_length=max_length))

if word in tokenizer.word_index:
    position = tokenizer.texts_to_sequences([word])
    print(f"Chỉ số của từ '{word}':", position[0][0])
    print("Chuỗi chứa chỉ số của từ:", position)
    embedding_of_word = modelx.predict(position)
    print(f"Embedding của từ '{word}':", embedding_of_word)

else:
    print("Từ '{}' không có trong từ điển.".format(word))

## Embedings Câu

In [ ]:
additional_test_sen = ["hà_nội t t chốt danh_sách dự afc cup thể_thao việt_nam"]
additional_test_seq = tokenizer.texts_to_sequences(additional_test_sen)
padded_additional_test_seq = pad_sequences(additional_test_seq, maxlen=max_length, truncating='post', padding='post')
print("Chuỗi chứa chỉ số của câu:", padded_additional_test_seq)

# Predict sentiment of the additional test sentence
modelx = Sequential()
modelx.add(Embedding(vocab_size, embedding_dim, input_length=max_length))

# Lấy embedding của chuỗi
embeddings = modelx.predict(padded_additional_test_seq)
print("Embeddings:", embeddings)